In [1]:
from functools import wraps

In [4]:
def func_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f"Log: {fn.__qualname__}({args}{kwargs}) = {result}")
        return result
    return inner

def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print("decorating", cls, name)
            setattr(cls, name, func_logger(obj))
    return cls

In [5]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def greet(self):
        return f"Hello, my name is {self.name} and I am {self.age} years old."

decorating <class '__main__.Person'> __init__
decorating <class '__main__.Person'> greet


In [6]:
Person("Alex", 10).greet()

Log: Person.__init__((<__main__.Person object at 0x0000018390686148>, 'Alex', 10){}) = None
Log: Person.greet((<__main__.Person object at 0x0000018390686148>,){}) = Hello, my name is Alex and I am 10 years old.


'Hello, my name is Alex and I am 10 years old.'

#### let's do the same with metaclass to decorate all our callables

In [8]:
class ClassLogger(type):
    def __new__(mcls, name, bases, class_dict):
        cls = super().__new__(mcls, name, bases, class_dict)
        for name, obj in vars(cls).items():
            if callable(obj):
                print("decorating", cls, name)
                setattr(cls, name, func_logger(obj))
        return cls

In [9]:
class Person(metaclass=ClassLogger):
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def greet(self):
        return f"Hello, my name is {self.name} and I am {self.age} years old."

decorating <class '__main__.Person'> __init__
decorating <class '__main__.Person'> greet


In [10]:
Person("Alex", 10).greet()

Log: Person.__init__((<__main__.Person object at 0x000001839069C588>, 'Alex', 10){}) = None
Log: Person.greet((<__main__.Person object at 0x000001839069C588>,){}) = Hello, my name is Alex and I am 10 years old.


'Hello, my name is Alex and I am 10 years old.'

why use metaclass and not decorators all the time? The decorating doesn't get inherited.

In [12]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def greet(self):
        return f"Hello, my name is {self.name} and I am {self.age} years old."

decorating <class '__main__.Person'> __init__
decorating <class '__main__.Person'> greet


In [13]:
class Student(Person):
    def __init__(self, name, age, number):
        super().__init__(name, age)
        self.student_number = number
    def study(self):
        return f"{self.name} studies..."

In [14]:
s = Student("Alex", 19, "1234")

Log: Person.__init__((<__main__.Student object at 0x00000183906A4588>, 'Alex', 19){}) = None


In [15]:
s.study()

'Alex studies...'

so we need to decorate Student class also

that's not the case with metaclass

In [16]:
class Person(metaclass=ClassLogger):
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def greet(self):
        return f"Hello, my name is {self.name} and I am {self.age} years old."

decorating <class '__main__.Person'> __init__
decorating <class '__main__.Person'> greet


In [17]:
class Student(Person):
    def __init__(self, name, age, number):
        super().__init__(name, age)
        self.student_number = number
    def study(self):
        return f"{self.name} studies..."

decorating <class '__main__.Student'> __init__
decorating <class '__main__.Student'> study


In [18]:
s = Student("Alex", 19, "1234")

Log: Person.__init__((<__main__.Student object at 0x00000183906AB108>, 'Alex', 19){}) = None
Log: Student.__init__((<__main__.Student object at 0x00000183906AB108>, 'Alex', 19, '1234'){}) = None


In [19]:
s.study()

Log: Student.study((<__main__.Student object at 0x00000183906AB108>,){}) = Alex studies...


'Alex studies...'

#### problem with inheritace with metaclasses:

In [23]:
class Metaclass1(type): #we inheriting __new__ from type
    pass
class Metaclass2(type):
    pass

In [24]:
class Person(metaclass=Metaclass1):
    pass

In [25]:
class Student(Person, metaclass=Metaclass2):
    pass

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

that means we cannot mix metaclasses in our inheritance chain

we can mix type and one metaclass

In [26]:
class CollegeStudent(Person, metaclass=type): #with type it's ok
    pass

we cannot mix two different custom metaclasses

In [27]:
class AccountHolder(metaclass=Metaclass2):
    pass

In [28]:
class BankAccountPerson(Person, AccountHolder):
    pass

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases